# The Python Data Model

This course is focused about the _protocol_ aspect of python.

## The Duke Typing orientation of Python

* There's no strict typing as _interface_ contract.
* Python prefers to use the _Duck Typing_ contract.

<div class="row">
<div class="col-md-8 col-md-offset-2">
<div class="alert alert-info text-center" style="font-size: larger;">
"If it walks like a duck and it quacks like a duck, then it must be a duck"
</div>
</div>
</div>

If a object has the requested functions **associated** to a *type* of **behavior**,  
it's sufficient to be considered as this *type*.

## Comparison with interface in java

It's very visible when compared to another language as java.

```java
public interface Bird {  //  <----------------- contract
    void fly();
}

pubic class BasicBird implements Bird {
    void fly() {
        System.out.println("Look Mom, I fly");
    }
}
```

To make a collection of `Bird` flying, we do

```java
    
// Very ugly iteration pattern, don't do this at home kids 
public static void makeThemFly(Object[] birds) {
    for(int i=0; i < birds.length; i++) {
        Bird bird = (Bird) birds[i];
        bird.fly();
    }
}
```

```java
...    
birds[0] = new BasicBird();
birds[1] = new BasicBird();
...
    
makeThemFly(birds);
```

Only sub-types of `Bird` could do the same

```java
public interface SeaBird extends Bird {
    void fish();
}

pubic class BasicSeaBird extends BasicBird implements SeaBird {
    void fish() {
        System.out.println("Look Dad, I fish");
    }
}
```


```java
...    
birds[0] = new BasicSeaBird();
birds[1] = new BasicSeaBird();
...
    
makeThemFly(birds);
```

Now let's talk about `Plane` which can also *fly*...


```java
public interface Plane {
    void fly();
}
```

`makeThemFly` will doesn't work with a `Plane`, because a `Plane` is **not** a `Bird` !

```java
Plane plane = new Plane();
Bird bird = (Bird) plane; // <--------------- ClassCastException !!!
```

The logic of *java*, as all strictly typed language, is the following :

* What it's your type ?
* Ok, in this type heritance, is there a method named *fly* ?
* Ok, I call it with parameters.

In *python*, we don't care !

In [ ]:
class Bird:
    def fly(self):
        print("Look Mom, I fly!")

class Plane:
    def fly(self):
        print("Look Sis, I fly!")


for s in [Bird(), Plane()]:
    s.fly()

The logic of *python* is the following:

* ~~What kind of Type are you ?~~
* ~~Ok, in this Type heritance,~~ is there a method named _fly_ ?
* Ok, I call it with parameters.

## Python Data Model

Python builds all its run engine upon this principe :

- Do you have the required _method_ with the required _name_ ?
- Does it accept the given parameters (number mainly) ?
- Ok, I call it.

**That's all.**

## Dunders

Python uses a **naming convention** marking some functions as specially used.

They're **prefixed** and **suffixed** with **double underscores** 

`__<name of function>__(parameters)`

They're called 
* *special method names* 
* or *dunders*

**double underscores** &rarr; **d**ouble **unders**cores &rarr; **dunders**

## Dunders examples

* `__init__` &rarr; class constructor
* `__str__` &rarr; return a human readable string representation of an object
* `__eq__`&rarr; equality operator

A complete list is available in [official documentation](https://docs.python.org/3/reference/datamodel.html#special-method-names) 

## Mapping

To each python operator is associated a **dunder**

* length : `len(o)` &rarr; `o.__len__()`
* addition: `a + b` &rarr; `a.__add__(b)`
* call: `f()` &rarr; `f.__call__()`
* ...

## More complex behaviors

More complex behaviors are associated to **several** dunders !

Do you know what's a **context manager** ?

The `with <something> as <thing>:`

```python
with open('/path/to/file','w') as file:
    file.write('Foo')
```

### Context Manager

Context manager is associated to these [dunders](https://docs.python.org/3/reference/datamodel.html#with-statement-context-managers)

* `object.__enter__(self)`(see [doc](https://docs.python.org/3/reference/datamodel.html#object.__enter__))
* `object.__exit__(self, exc_type, exc_value, traceback)`(see [doc](https://docs.python.org/3/reference/datamodel.html#object.__exit__))

It's very useful because it **never forgets to free or close used resources**. 

```
WITH <return context manager> AS <context manager.__enter__()> :
    # do something with AS object
    
# Before exit of WITH clause, call <context manager.__exit__()>
```

Example with a context manager locking instrument and returning a buffer to store image.

```python
with Instrument.lock('VIS') as buffer:
    buffer.append('observation.2019.01.01.fit')
```

In [ ]:
REGISTRY = {'VIS': {'status': 'available'}, # <---------- Registry of all Instruments (2...)
            'NIR': { 'status': 'available'}}

class Instrument:
    
    def __init__(self,code):
        self.code = code
        self.buffer = []
    
    @staticmethod
    def lock(code):         # <-------------------------- the WITH part
        if REGISTRY[code]['status'] == 'available':
            REGISTRY[code]['status']='engaged'
            return Instrument(code)
        
        raise ValueError(f'{code} is not available')
        
    def __enter__(self):          #<------------------------- the AS part
        return self.buffer
    
    def __exit__(self, exc_type, exc_value, traceback):#<---- the after block part
        REGISTRY[self.code]['status'] = 'available'
        

The same with some print statements...

In [ ]:
class Instrument:
    
    def __init__(self,code):
        self.code = code
        self.buffer = []
    
    @staticmethod
    def lock(code):
        if REGISTRY[code]['status'] == 'available':
            print(f'Observation on {code} granted') #<---------------------- Granted
            REGISTRY[code]['status']='engaged'
            return Instrument(code)
        
        raise ValueError(f'{code} is not available')
        
    def __enter__(self):
        print(f'   OPEN Buffer on {self.code}') #<--------------------------- open and return buffer
        return self.buffer
    
    def __exit__(self, exc_type, exc_value, traceback):
        REGISTRY[self.code]['status'] = 'available'
        print(f'   CLOSE buffer {self.buffer} on {self.code}') #<------------ Close buffer
        

In [ ]:
print(f'Before WITH => {REGISTRY}')

with Instrument.lock('VIS') as buffer:
    print(f'   ... In With => {REGISTRY}')
    buffer.append('observation.2019.01.01.fit')
    
print(f'After WITH => {REGISTRY}')


## Subscriptable object

If you would get a class that could be used as a dict.

* `object.__getitem__`
* `object.__setitem__`

In [ ]:
class Observation:
    
    def __init__(self, configuration, data):
        self.configuration = configuration
        self.data = data

In [ ]:
obs = Observation({'date': '2019-01-01', 'temperature': '286,15K'}, data=[12.0, 14.0, 15.0])

obs['date'] == '2019-01-01'

To make it **subscriptable** add a `__getitem__` method

In [ ]:
class Observation:
    
    def __init__(self, configuration, data):
        self.configuration = configuration
        self.data = data
        
    def __getitem__(self, key):
        if key == 'data':
            return self.data
        return self.configuration[key]

In [ ]:
obs = Observation({'date': '2019-01-01', 'temperature': '286,15K'}, data=[12.0, 14.0, 15.0])

assert obs['date'] == '2019-01-01'

## Create operators

It looks very scholar (we all learned this)

* `+` &rarr; `__add___`
* `*` &rarr; `__multiply___`

Imagine we want to add data to observation's data as 

In [ ]:
obs = Observation({'date': '2019-01-01', 'temperature': '286,15K'}, 
                  data=[12.0, 14.0, 15.0])

obs + [15.6, 8.5] == [12.0, 14.0, 15.0, 15.6, 8.5]

We have to implement `__add__`

In [ ]:
class Observation:
    
    def __init__(self, configuration, data):
        self.configuration = configuration
        self.data = data
        
    def __add__(self,o):
        
        res = NotImplemented #<------------ mandatory (see next) !! 
        
        if isinstance(o, list):
            self.data.extend(o) #<--------- list extension
            res = self.data
        elif isinstance(o, float):
            self.data.append(o) #<---------- list append
            res = self.data
        return res

In [ ]:
obs = Observation({'date': '2019-01-01', 'temperature': '286,15K'}, data=[12.0, 14.0, 15.0])

assert obs + [15.6, 8.5] == [12.0, 14.0, 15.0, 15.6, 8.5]

And what if we want the inversed order operation ?

```list + observation -> list```

In [ ]:
obs = Observation({'date': '2019-01-01', 'temperature': '286,15K'}, data=[12.0, 14.0, 15.0])

assert [15.6, 8.5] + obs == [12.0, 14.0, 15.0, 15.6, 8.5]

We have to implement `__radd__`, the *right-hand* addition.

Python does the following for `a + b`

* it tries `a.__add__(b)`
* if it **returns** `NotImplemented`, it tries `b.__radd__(a)`

That's why returning `NotImplemented` is **mandatory**.

In [ ]:
class Observation:
    
    def __init__(self, configuration, data):
        self.configuration = configuration
        self.data = data
        
    def __add__(self,o):
        
        res = NotImplemented #<------------ mandatory !! 
        
        if isinstance(o, list):
            self.data.extend(o)
            res = self.data
        elif isinstance(o, float):
            self.data.append(o)
            res = self.data
        return res
    
    def __radd__(self,o):
        return self.__add__(o)

In [ ]:
obs = Observation({'date': '2019-01-01', 'temperature': '286,15K'}, data=[12.0, 14.0, 15.0])

assert [15.6, 8.5] + obs == [12.0, 14.0, 15.0, 15.6, 8.5]

# Conclusion

The dunders are at the heart of python.

Don't take them for some *syntaxic sugars* and *funny toys*.

That's how Python works, and if you want to hack it, take a look at them.